# Music Genre Prediction

In [1]:
class Config:
    NB = '205'
    dataset_NB = '103'
    #stacking_NB = ['212', '213', '214']
    stacking_NB = False

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'index'
    target = 'genre'

## Import libralies

In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)

color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
    'Cat10': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E', '#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
import random
import joblib
import itertools
from itertools import combinations
from imblearn import FunctionSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from sklearn.metrics import roc_auc_score, roc_curve, auc, f1_score, confusion_matrix
import scipy.stats as stats
from lightgbm import LGBMClassifier, early_stopping

## Load and check data

In [5]:
df_train = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_train.pkl', compression='zip')
df_test = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_test.pkl', compression='zip')

submission = pd.read_csv(Config.raw_data_dir + 'sample_submit.csv', header=None)

df_train.shape

(4046, 285)

In [6]:
df_train.head()

,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_int,region_A,region_B,region_C,region_D,region_E,region_F,region_G,region_H,region_I,region_J,region_K,region_L,region_M,region_N,region_O,region_P,region_Q,region_R,region_S,region_T,unknown,duration_long,popularity_add_duration_ms,popularity_sub_duration_ms,popularity_mul_duration_ms,popularity_bigger_duration_ms,popularity_add_acousticness,popularity_sub_acousticness,popularity_mul_acousticness,popularity_bigger_acousticness,popularity_add_positiveness,popularity_sub_positiveness,popularity_mul_positiveness,popularity_bigger_positiveness,popularity_add_danceability,popularity_sub_danceability,popularity_mul_danceability,popularity_bigger_danceability,popularity_add_loudness,popularity_sub_loudness,popularity_mul_loudness,popularity_bigger_loudness,popularity_add_energy,popularity_sub_energy,popularity_mul_energy,popularity_bigger_energy,popularity_add_liveness,popularity_sub_liveness,popularity_mul_liveness,popularity_bigger_liveness,popularity_add_speechiness,popularity_sub_speechiness,popularity_mul_speechiness,popularity_bigger_speechiness,popularity_add_instrumentalness,popularity_sub_instrumentalness,popularity_mul_instrumentalness,popularity_bigger_instrumentalness,popularity_add_tempo_int,popularity_sub_tempo_int,popularity_mul_tempo_int,popularity_bigger_tempo_int,duration_ms_add_acousticness,duration_ms_sub_acousticness,duration_ms_mul_acousticness,duration_ms_bigger_acousticness,duration_ms_add_positiveness,duration_ms_sub_positiveness,duration_ms_mul_positiveness,duration_ms_bigger_positiveness,duration_ms_add_danceability,duration_ms_sub_danceability,duration_ms_mul_danceability,duration_ms_bigger_danceability,duration_ms_add_loudness,duration_ms_sub_loudness,duration_ms_mul_loudness,duration_ms_bigger_loudness,duration_ms_add_energy,duration_ms_sub_energy,duration_ms_mul_energy,duration_ms_bigger_energy,duration_ms_add_liveness,duration_ms_sub_liveness,duration_ms_mul_liveness,duration_ms_bigger_liveness,duration_ms_add_speechiness,duration_ms_sub_speechiness,duration_ms_mul_speechiness,duration_ms_bigger_speechiness,duration_ms_add_instrumentalness,duration_ms_sub_instrumentalness,duration_ms_mul_instrumentalness,duration_ms_bigger_instrumentalness,duration_ms_add_tempo_int,duration_ms_sub_tempo_int,duration_ms_mul_tempo_int,duration_ms_bigger_tempo_int,acousticness_add_positiveness,acousticness_sub_positiveness,acousticness_mul_positiveness,acousticness_bigger_positiveness,acousticness_add_danceability,acousticness_sub_danceability,acousticness_mul_danceability,acousticness_bigger_danceability,acousticness_add_loudness,acousticness_sub_loudness,acousticness_mul_loudness,acousticness_bigger_loudness,acousticness_add_energy,acousticness_sub_energy,acousticness_mul_energy,acousticness_bigger_energy,acousticness_add_liveness,acousticness_sub_liveness,acousticness_mul_liveness,acousticness_bigger_liveness,acousticness_add_speechiness,acousticness_sub_speechiness,acousticness_mul_speechiness,acousticness_bigger_speechiness,acousticness_add_instrumentalness,acousticness_sub_instrumentalness,acousticness_mul_instrumentalness,acousticness_bigger_instrumentalness,acousticness_add_tempo_int,acousticness_sub_tempo_int,acousticness_mul_tempo_int,acousticness_bigger_tempo_int,positiveness_add_danceability,positiveness_sub_danceability,positiveness_mul_danceability,positiveness_bigger_danceability,positiveness_add_loudness,positiveness_sub_loudness,positiveness_mul_loudness,positiveness_bigger_loudness,positiveness_add_energy,positiveness_sub_energy,positiveness_mul_energy,positiveness_bigger_energy,positiveness_add_liveness,positiveness_sub_liveness,positiveness_mul_liveness,positiveness_bigger_liveness,positiveness_add_speechiness,positiveness_sub_speechiness,positiveness_mul_speechiness,positiveness_bigger_speechiness,positiveness_add_instrumentalness,positiveness_sub_instrumentalness,pos

In [7]:
for i in df_train.columns:
    print(i)

index
genre
popularity
duration_ms
acousticness
positiveness
danceability
loudness
energy
liveness
speechiness
instrumentalness
tempo_int
region_A
region_B
region_C
region_D
region_E
region_F
region_G
region_H
region_I
region_J
region_K
region_L
region_M
region_N
region_O
region_P
region_Q
region_R
region_S
region_T
unknown
duration_long
popularity_add_duration_ms
popularity_sub_duration_ms
popularity_mul_duration_ms
popularity_bigger_duration_ms
popularity_add_acousticness
popularity_sub_acousticness
popularity_mul_acousticness
popularity_bigger_acousticness
popularity_add_positiveness
popularity_sub_positiveness
popularity_mul_positiveness
popularity_bigger_positiveness
popularity_add_danceability
popularity_sub_danceability
popularity_mul_danceability
popularity_bigger_danceability
popularity_add_loudness
popularity_sub_loudness
popularity_mul_loudness
popularity_bigger_loudness
popularity_add_energy
popularity_sub_energy
popularity_mul_energy
popularity_bigger_energy
popularity_add

## Stacking Setting

In [8]:
if Config.stacking_NB is False:
    print('stacking is not setting.')
else:
    for i in Config.stacking_NB:
        df_train_NB =  pd.read_csv(Config.interim_dir + f'nb{i}.csv')
        df_test_NB =  pd.read_csv(Config.submission_dir + f'nb{i}.csv')

        df_train[f'nb{i}'] = df_train_NB[f'nb{i}']

        df_test = df_test.reset_index()
        df_test[f'nb{i}'] = df_test_NB[Config.target]
        df_test = df_test.set_index('index')

    df_test

stacking is not setting.


## Training

In [9]:
def seed_everything(seed):

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [10]:
# Get feature list
features = [col for col in df_train.columns if col not in [Config.row_id, Config.target]]

# Get parameter list
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 11,
    'metric': 'multi_logloss',
    'seed': Config.random_seed,
    'n_estimators': 20000,
    'max_depth': -1,
    'num_leaves': 32,
    'learning_rate': 0.01,
    #'feature_fraction': 0.20,
    #'bagging_freq': 10,
    #'bagging_fraction': 0.95,
    'n_jobs': -1,
    #'lambda_l2': 2,
    'min_data_in_leaf': 5,
}

callbacks = [lgb.early_stopping(50), lgb.log_evaluation(50)]

In [11]:
# 約10秒

# Create a numpy array to store test predictions
test_predictions = np.zeros((len(df_test), Config.n_folds))

# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(df_train))

feature_importance_df = pd.DataFrame(index=features)
y_valids, val_preds =[],[]

kfold = StratifiedKFold(n_splits=Config.n_folds, shuffle=True, random_state=Config.random_seed)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(df_train, df_train[Config.target])):

    print(' ')
    print('-'*50)
    print(f'Training fold {fold+1} with {len(features)} features...')

    X_train, X_val = df_train[features].iloc[train_idx], df_train[features].iloc[valid_idx]
    y_train, y_val = df_train[Config.target].iloc[train_idx], df_train[Config.target].iloc[valid_idx]

    # Over Sampling
    sm = SMOTE(random_state=Config.random_seed)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    # print(y_train.value_counts())

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_val, y_val)

    model = lgb.train(params=params, train_set=lgb_train, valid_sets=[lgb_train, lgb_valid], valid_names=['train', 'valid'], callbacks=callbacks)
    print(f'================================== training {fold+1} fin. ==================================')

    # Predict validation data
    print(f'================================== validation-data predicting ... ==================================')
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    val_pred = np.argmax(val_pred, axis=1)
    oof_predictions[valid_idx] = val_pred

    # Predict test data
    print(f'================================== test-data predicting ... ==================================')
    test_pred = model.predict(df_test[features], num_iteration=model.best_iteration)
    test_pred = np.argmax(test_pred, axis=1)

    test_predictions[:, fold] += test_pred

    # save results
    y_valids.append(y_val)
    val_preds.append(val_pred)
    feature_importance_df["Importance_Fold"+str(fold+1)]=model.feature_importance(importance_type='gain')

    # Compute fold metric
    val_pred = pd.DataFrame(data={'prediction': val_pred})
    y_val = pd.DataFrame(data={'target': y_val.reset_index(drop=True)})
    score = f1_score(y_val, val_pred, average='macro')

    print(f'Fold {fold+1} CV result')
    print(f'metric : {score}')

    del X_train, X_val, y_train, y_val, lgb_train, lgb_valid
    _ = gc.collect()

# Compute out of folds metric
oof_predictions = pd.DataFrame(data={'prediction': oof_predictions})
y_true = pd.DataFrame(data={Config.target: df_train[Config.target]})

print(' ')
print('-'*50)
print(f'TOTAL socre : {f1_score(df_train[Config.target], oof_predictions["prediction"], average="macro")}')
print('-'*50)

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({Config.row_id: df_train[Config.row_id], Config.target: df_train[Config.target], 'prediction': oof_predictions['prediction']})

# Create a dataframe to store test prediction


test_predictions, _ = stats.mode(test_predictions, axis=1)
test_predictions = test_predictions.reshape(-1)

test_df = pd.DataFrame({Config.row_id: df_test[Config.row_id], Config.target: test_predictions})

 
--------------------------------------------------
Training fold 1 with 283 features...
10.0    1069
8.0     1069
3.0     1069
7.0     1069
2.0     1069
1.0     1069
5.0     1069
9.0     1069
6.0     1069
4.0     1069
0.0     1069
Name: genre, dtype: int64
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 55803
[LightGBM] [Info] Number of data points in the train set: 11759, number of used features: 272
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Info] Start training from score -2.397895
[LightGBM] [Inf

In [12]:
oof_df.head()

,index,genre,prediction
0,0,10.0,10.0
1,1,8.0,8.0
2,2,3.0,8.0
3,3,10.0,10.0
4,4,3.0,3.0


In [13]:
# Save results
oof_df_tmp = oof_df.drop(columns=[Config.target])
oof_df_tmp.columns = [Config.row_id, f'nb{Config.NB}']
oof_df_tmp.to_csv(Config.interim_dir + f'nb{Config.NB}.csv', index=False)
oof_df_tmp

,index,nb205
0,0,10.0
1,1,8.0
2,2,8.0
3,3,10.0
4,4,3.0
...,...,...
4041,4041,8.0
4042,4042,10.0
4043,4043,8.0
4044,4044,10.0


In [14]:
cm = confusion_matrix(oof_df[Config.target], oof_df['prediction'], normalize='true')

names = [f'Target_{i}' for i in range(11)]

fig = ff.create_annotated_heatmap(cm, x=names, y=names)
fig.update_layout(
    yaxis_title='True Label',
    xaxis_title='Pred Label',
)
fig.show()

In [29]:
top = 50

feature_importance_df['avg'] = feature_importance_df.mean(axis=1)
feature_importance_top = feature_importance_df.avg.nlargest(top).sort_values(ascending=True)

pal=sns.color_palette("YlGnBu_r", top).as_hex()
fig=go.Figure()
for i in range(len(feature_importance_top.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feature_importance_top[i],
                       line_color=pal[::-1][i],opacity=0.8,line_width=4))

fig.add_trace(go.Scatter(x=feature_importance_top, y=feature_importance_top.index, mode='markers',
                         marker_color=pal[::-1], marker_size=8,
                         hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))

fig.update_layout(template=plotly_template,title=f'LGBM Feature Importance<br>Top {top}',
                  margin=dict(l=300,t=80),
                  xaxis=dict(title='Importance', zeroline=False),
                  yaxis_showgrid=False, height=1000, width=800)
fig.show()

In [21]:
feature_importance_top

popularity_bigger_liveness                1126.323485
liveness_bigger_instrumentalness          1171.554492
region_A                                  1185.781283
popularity_bigger_positiveness            1213.871846
popularity_bigger_loudness                1244.416965
positiveness_bigger_energy                1361.047917
duration_ms_bigger_energy                 1377.527354
acousticness_bigger_speechiness           1400.314439
danceability_bigger_loudness              1518.423155
popularity_bigger_instrumentalness        1600.115276
positiveness                              1627.759378
positiveness_bigger_instrumentalness      1678.478603
tempo_int                                 1817.052714
acousticness_bigger_liveness              1921.625901
liveness_add_tempo_int                    1924.067120
energy_add_liveness                       1995.491530
acousticness_bigger_danceability          2015.224224
danceability_bigger_liveness              2050.873530
liveness_bigger_speechiness 

In [16]:
fig = go.Figure(layout=plotly_template['layout'])
fig.add_trace(
    go.Histogram(
        x=test_df[Config.target],
        name=f'Prediction',
        histnorm='probability',
        marker=dict(color=color_palette['Bin'][0]),
        #line=dict(color='black')
    ),
)

fig.add_trace(
    go.Histogram(
        x=df_train[Config.target],
        name=f'Train',
        histnorm='probability',
        marker=dict(color=color_palette['Bin'][1]),
        opacity=0.5
        #line=dict(color='black')
    ),
)

fig.update_layout(
    title='Prediction Distribution',
    barmode='overlay',
    uniformtext_minsize=15,
    uniformtext_mode='hide',
    width=700)

fig.show()

In [ ]:
test_df = test_df.astype({'genre': int})
test_df.info()

In [ ]:
test_df[Config.target].describe()

In [ ]:
Config.NB

In [ ]:
test_df.to_csv(Config.submission_dir + f'nb{Config.NB}.csv', index=False, header=False)

## 検証メモ